In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [394]:
df = pd.read_csv('moons.csv')
df = df.iloc[:500, :]

In [395]:
from scipy.spatial.distance import pdist, squareform

distances = squareform(pdist(df, metric='euclidean'))

In [396]:
import heapq
from tqdm import tqdm

n = df.shape[0]
dist_pr = []

for i in tqdm(range(n), desc="Processing Distances", unit="i"):
    for j in range(i + 1, n):
        if i == j:
            continue
        current_dist = distances[i, j]
        heapq.heappush(dist_pr, (current_dist, [i, j]))


Processing Distances: 100%|██████████| 500/500 [00:00<00:00, 1030.06i/s]


In [397]:
distances = pd.DataFrame(distances)

In [398]:
shared_dict = {i: [i] for i in range(0, n)}

In [399]:
len(shared_dict)

500

In [400]:
# n = 500
zeros = np.zeros((n-1, 3), dtype=int) 
empty_dicts = np.array([{}] * (n-1), dtype=object)
join_df = np.column_stack((zeros, empty_dicts))

In [401]:
merged = [0 for i in range(n+1)]

In [402]:
distances1 = distances.copy()

In [403]:
from tqdm import tqdm
import heapq
import numpy as np

counter = 0
merged_counter = 1

total_merges = n - 1

with tqdm(total=total_merges, desc="Merging Points") as pbar:
    while len(dist_pr) > 1:
        min_pts = heapq.heappop(dist_pr)
        point_A = min_pts[1][0]
        point_B = min_pts[1][1]
        point_distance = min_pts[0]

        if merged[point_A] == 1 or merged[point_B] == 1:
            continue

        merged[point_A] = 1
        merged[point_B] = 1
        merged = np.append(merged, 0)
        new_index = n + merged_counter
        new_dists = []

        for index, row in distances.iterrows():
            dist_AC = distances.at[point_A, index]  
            dist_BC = distances.at[point_B, index] 
            dist_ABC = min(dist_AC, dist_BC) 

            new_dists.append(dist_ABC)


            heapq.heappush(dist_pr, (dist_ABC, [index, new_index]))

        distances[new_index] = new_dists
        distances.loc[new_index] = new_dists + [0]
        distances.drop(index=[point_A, point_B], columns=[point_A, point_B], inplace=True)

        shared_dict[new_index] = shared_dict[point_A] + shared_dict[point_B]
        if point_A in shared_dict:
            shared_dict.pop(point_A)
        if point_B in shared_dict:
            shared_dict.pop(point_B)

        join_df[counter] = [point_A, point_B, point_distance, shared_dict.copy()]
        counter += 1
        merged_counter += 1

        pbar.update(1)

Merging Points:  33%|███▎      | 167/499 [00:08<00:12, 26.76it/s]/tmp/ipykernel_19380/1593408466.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances[new_index] = new_dists
/tmp/ipykernel_19380/1593408466.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances[new_index] = new_dists
/tmp/ipykernel_19380/1593408466.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

In [417]:
from sklearn.metrics import silhouette_score

k_vals = [2, 5, 10]
scores = []

for k in k_vals:
    d = join_df[join_df.shape[0]-(k)][3]
    df['clusters'] = [0] * n

    for key in d:
        df.iloc[d[key], 2] = key
    scores.append(silhouette_score(df, df['clusters']))

In [418]:
scores

[0.49469557835542854, 0.6465838147320061, 0.6147622760409484]